# Librairies et fichiers

In [1]:
import numpy as np
import pandas as pd

import scipy
from scipy import ndimage
from scipy import signal
from pylab import *
import time
from time import time
import math
import random 

import pywt
import pywt.data

import os

from copy import deepcopy

In [2]:
mit_train = pd.read_csv('mitbih_train.csv', header = None)
mit_test = pd.read_csv('mitbih_test.csv', header = None)

ptbdb_normal = pd.read_csv('ptbdb_normal.csv', header=None)
ptbdb_abnormal = pd.read_csv('ptbdb_abnormal.csv', header=None)

# constitution du jeu concaténation des jeux de données initiaux
ptbdb = pd.concat([ptbdb_normal, ptbdb_abnormal])
mit = pd.concat([mit_train, mit_test])

In [3]:
X_ptb = ptbdb.iloc[:,:-1]
X_test = mit_test.iloc[:,:-1]
X_train = mit_train.iloc[:,:-1]
y_ptb = ptbdb.iloc[:,-1]
y_test = mit_test.iloc[:,-1]
y_train = mit_train.iloc[:,-1]

In [ ]:
targets = mit_train.iloc[:,-1]
for elem in targets :
    targets[elem]=int(targets[elem])

In [56]:
targets.values_count()

AttributeError: 'Series' object has no attribute 'values_count'

# Fonctions

In [ ]:
# liste du nombre de lignes associées à chaque item des dataframe
def l_dim (signaux) :
    # prend en entrée un dataframe
    # retourne la liste du nombre d'élement pour chaque cible du dataframe
    

In [4]:
#isoler le dataframe signaux : suprimer les autres colonnes...

def get_cwt_line (signaux, scale, wave, line):
    # prend en entrée le jeu de données, une valeur d'échelle, un type d'ondelettes continues et une classe de signaux
    # renvoie une matrice de wavelett continues correpondant à l'entrée numéro line
    # attention, la dernière ligne de signaux contient le target
    m=signaux.shape[1] # nombre de col
    widths = np.arange(1,scale+1) # définie un intervalle à partir de l'échelle
    cwtmatr, freqs = pywt.cwt(signaux, widths, wave)
    return (signaux.iloc[line,:],freqs, cwtmatr[:,line,:-1])

In [5]:
def redim_array (original_array, width, height) :
    # prend en entrée un array avec les des  dims avec lesquelles ont veut le redimentionner
    # renvoie l'array redimentionné

#    original_array = matr
 #   width , height = 28, 28
    resize_array = np.zeros(shape=(width,height))

    for W in range(width):
        for H in range(height):
            new_width = int( W * original_array.shape[0] / width )
            new_height = int( H * original_array.shape[1] / height )
            resize_array[W][H] = original_array[new_width][new_height]

    return(resize_array)

# Selection des paramètres

In [6]:
scale = 28 # échelle faible pour limiter le temps de calcul
cw_type = 'mexh' # type d'ondelettes continues séléctionnable parmi la liste ci-dessous

print(pywt.wavelist(kind='continuous'))

['cgau1', 'cgau2', 'cgau3', 'cgau4', 'cgau5', 'cgau6', 'cgau7', 'cgau8', 'cmor', 'fbsp', 'gaus1', 'gaus2', 'gaus3', 'gaus4', 'gaus5', 'gaus6', 'gaus7', 'gaus8', 'mexh', 'morl', 'shan']


# Pre-Processing

In [7]:
h = scale # pour avoir des images carrées qui puissent tourner dans les algorihmes CNN

In [8]:
t0 = time()

iMax = 641 # pour chaque valeur de cible 
train_len = iMax * 5
#iMax=len(data_brut), il faudra trouver la valeur limite correcte pour toutes les valeurs de k
n=5 #  5 vals possible pour target avec le fichier mit

width , height = scale, h #: dimension de l'image
col = range(0, width*height+1)
data_pre_ana = pd.DataFrame(columns = col)
for k in range(0,n) :
 #   print(k)
    data_brut = mit_train[mit_train.iloc[:,-1]==k] # DataFrame d'entrée # DataFrame d'entrée
    signaux=data_brut.iloc[:,:-1]  # Selectionner la partie signaux de 0 à 186 

    #Boucle sur les lignes de data_brut
    for i in range(0, iMax) :
  #      print(i)
        #Traiter par ondelette le signal (fonction get_cwt)
        sig, freq, matr = get_cwt_line (signaux, scale, cw_type, i)
        
        #Traiter la matrice pour l'inclure dans l'array de taille prédéfinie adaptée au code CNN 
        original_array = matr
       # print(matr)
        matr=redim_array(original_array, width, height)
        matr_1D = matr.flatten()  # on transforme l amatrice en array 1D 
        
        # ajouter y à la fin de matr_1D, 
       # print(data_brut.shape[1])
        y = data_brut.iloc[i,data_brut.shape[1]-1]
        matr_y = np.append(matr_1D, y)
        
        # Enregistrer dans data_pre_ana
        a_series = pd.Series(matr_y)
        data_pre_ana = data_pre_ana.append(a_series, ignore_index=True)
        
t1 = time() - t0
print("Réalisé en {} secondes".format(round(t1,3)))

In [9]:
nomFicTrain = 'cwt_{}x{}x{}_{}_train.csv'.format(scale, h, train_len, cw_type)
data_pre_ana.to_csv(nomFicTrain)
print('Fichier',nomFicTrain,'crée')

Fichier cwt_28x28x125_mexh_train.csv crée


In [10]:
X_train = data_pre_ana.iloc[:,:-1]
y_train = data_pre_ana.iloc[:,-1]

# Shape of X_train and y_train
print('Shape of X:', X_train.shape)
print('Shape of y:', y_train.shape)

Shape of X: (125, 784)
Shape of y: (125,)


In [11]:
test_size = 0.3

In [18]:
t0 = time()

iMax = int(iMax*test_size)
test_len = iMax * 5
#iMax=len(data_brut), il faudra trouver la valeur limite correcte pour toutes les valeurs de k
n=5 #  5 vals possible pour target
#prépparer le dataframe pour les résultats

width , height = scale, h #: dimension de l'image
col = range(0, width*height+1)
data_pre_ana = pd.DataFrame(columns = col)
for k in range(0,n) :
 #   print(k)
    data_brut = mit_test[mit_test.iloc[:,-1]==k] # DataFrame d'entrée # DataFrame d'entrée
    signaux=data_brut.iloc[:,:-1]  # Selectionner la partie signaux de 0 à 186 

    #Boucle sur les lignes de data_brut
    for i in range(0, iMax) :
  #      print(i)
        #Traiter par ondelette le signal (fonction get_cwt)
        sig, freq, matr = get_cwt_line (signaux, scale, cw_type, i)
        
        #Traiter la matrice pour l'inclure dans l'array de taille prédéfinie adaptée au code CNN 
        original_array = matr
       # print(matr)
        matr=redim_array(original_array, width, height)
        matr_1D = matr.flatten()  # on transforme l amatrice en array 1D 
        
        # ajouter y à la fin de matr_1D, 
       # print(data_brut.shape[1])
        y = data_brut.iloc[i,data_brut.shape[1]-1]
        matr_y = np.append(matr_1D, y)
        
        # Enregistrer dans data_pre_ana
        a_series = pd.Series(matr_y)
        data_pre_ana = data_pre_ana.append(a_series, ignore_index=True)
        
t1 = time() - t0
print("Réalisé en {} secondes".format(round(t1,3)))

KeyboardInterrupt: 

In [13]:
nomFicTest = 'cwt_{}x{}x{}_{}_test.csv'.format(scale, h, test_len, cw_type)
data_pre_ana.to_csv(nomFicTest)
print('Fichier',nomFicTest,'crée')

Fichier cwt_28x28x35_mexh_test.csv crée


In [15]:
X_test = data_pre_ana.iloc[:,:-1]
y_test = data_pre_ana.iloc[:,-1]

# Shape of X_test and y_test
print('Shape of X:', X_test.shape)
print('Shape of y:', y_test.shape)

Shape of X: (0, 784)
Shape of y: (0,)
